In [1]:
import time
import requests
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as soup

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import re

In [4]:
browser = Browser('edge')

In [5]:
url = "https://www.canadiantire.ca/en/promotions/clearance.html?page=1"
browser.visit(url)

In [6]:
html = browser.html
html_soup = soup(html, 'html.parser')

In [9]:
product_info = html_soup.find_all(class_ = 'nl-product-card__primary-details')

print(product_info)

[<div class="nl-product-card__primary-details"><div aria-hidden="true" class="nl-product-card__image-wrap"><img alt="Bluehive Mouse Pad with Wireless Charger" class="" data-component-name="product-card" data-src="https://media-www.canadiantire.ca/product/automotive/car-care-accessories/auto-electronics/0358045/bluehive-mouse-pad-with-wireless-charging-4a422bdb-cc9c-4daf-821a-689c9cea2af5-jpgrendition.jpg?im=whresize&amp;wid=142&amp;hei=142" src="https://media-www.canadiantire.ca/product/automotive/car-care-accessories/auto-electronics/0358045/bluehive-mouse-pad-with-wireless-charging-4a422bdb-cc9c-4daf-821a-689c9cea2af5-jpgrendition.jpg?im=whresize&amp;wid=200&amp;hei=200"/></div><div class="nl-product-card__product-information"><div class="nl-product"><div class="nl-product-title-sku"><div class="nl-product-card__title nl-product--trim" id="title__promolisting-035-8045-4">Bluehive Mouse Pad with Wireless Charger</div></div></div><div aria-hidden="true" class="nl-reviews__list" data-bv

In [8]:
product_name = []
price = []
clearance_price = []

for product in product_info:
    product.append(product.div['class="nl-product-card__title nl-product--trim"'])

KeyError: 'class="nl-product-card__title nl-product--trim"'

In [4]:
time.sleep(10)
try:
    # Find the "Allow" button and click it if the popup appears
    allow_button = driver.find_element_by_xpath("//button[@aria-label='Allow']")
    allow_button.click()
except:
    # If the popup does not appear or the "Allow" button is not found, move on.
    pass

C:\Users\krist\AppData\Local\Temp\ipykernel_6492\3050904393.py:4: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  allow_button = driver.find_element_by_xpath("//button[@aria-label='Allow']")


In [12]:
# Extract product name
product_name = soup.find('div', class_='nl-product-card__title nl-product--trim').text.strip()

# Extract original price
origin_price_div = soup.find('span', class_='nl-price--total--red')
original_price = origin_price_div.text.strip() if origin_price_div else "N/A"

# Extract clearance price
clearance_price_span = soup.find('span', class_='nl-price--total--red', data_testid='priceTotal')
clearance_price = clearance_price_span.text.strip()

print("Product Name:", product_name)
print("Original Price:", original_price)
print("Clearance Price:", clearance_price)

AttributeError: 'NoneType' object has no attribute 'text'